In [357]:
import pandas as pd
import numpy as np

In [358]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.pop('Id')
id=test.pop('Id')
y = ["SalePrice"]
label=train["SalePrice"]

In [359]:
#给变量分类
num_columns = []
cate_columns = []
for column in test.columns:
    if test.dtypes[column] != np.dtype('object'):
        num_columns.append(column)
    else:
        cate_columns.append(column)
print(num_columns)
print(cate_columns)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCon

In [360]:
# ## 筛选预测能力强的变量

# + 连续变量筛选-相关性

corr_s = abs(train[y + num_columns].corr(method = 'spearman'))
corr_s = pd.DataFrame(corr_s.iloc[0,:])

corr_p = abs(train[y + num_columns].corr(method = 'pearson'))
corr_p = pd.DataFrame(corr_p.iloc[0,:])

corr_sp = pd.concat([corr_s,corr_p], axis = 1)
corr_sp.columns = ['spearman','pearson']

corr_sp[(corr_sp['spearman'] <= 0.1) & (corr_sp['pearson'] <= 0.1)]

,spearman,pearson
MSSubClass,0.007192,0.084284
BsmtFinSF2,0.038806,0.011378
LowQualFinSF,0.067719,0.025606
BsmtHalfBath,0.012189,0.016844
3SsnPorch,0.065440,0.044584
PoolArea,0.058453,0.092404
MiscVal,0.062727,0.021190
MoSold,0.069432,0.046432
YrSold,0.029899,0.028923


In [361]:
#剔除掉相关性不大的变量
num_columns = set(num_columns) - set(['MSSubClass','BsmtFinSF2',
                            'LowQualFinSF','BsmtHalfBath',
                            '3SsnPorch','PoolArea','MiscVal','MoSold','YrSold'])
num_columns = list(num_columns)
len(num_columns)

27

In [362]:
# + 分类变量筛选-方差分析
import statsmodels.stats.anova as anova
from statsmodels.formula.api import ols

for i in cate_columns:
    formula = "SalePrice ~ " + str(i)
    print((anova.anova_lm(ols(formula,data = train[cate_columns+['SalePrice']]).fit()).F))

MSZoning    43.840282
Residual          NaN
Name: F, dtype: float64
Street      2.45929
Residual        NaN
Name: F, dtype: float64
Alley       35.56206
Residual         NaN
Name: F, dtype: float64
LotShape    40.132852
Residual          NaN
Name: F, dtype: float64
LandContour    12.850188
Residual             NaN
Name: F, dtype: float64
Utilities    0.298804
Residual          NaN
Name: F, dtype: float64
LotConfig    7.809954
Residual          NaN
Name: F, dtype: float64
LandSlope    1.958817
Residual          NaN
Name: F, dtype: float64
Neighborhood    71.784865
Residual              NaN
Name: F, dtype: float64
Condition1    6.118017
Residual           NaN
Name: F, dtype: float64
Condition2    2.073899
Residual           NaN
Name: F, dtype: float64
BldgType    13.011077
Residual          NaN
Name: F, dtype: float64
HouseStyle    19.595001
Residual            NaN
Name: F, dtype: float64
RoofStyle    17.805497
Residual           NaN
Name: F, dtype: float64
RoofMatl    6.727305
Residual 

In [363]:
cate_columns = list(set(cate_columns) - set(['MiscFeature','Fence','PoolQC','GarageCond','GarageQual','Functional','Heating','BsmtFinType2',
                                             'ExterCond','RoofMatl','Condition2','Condition1','LandSlope','LotConfig','Utilities','Street','LandContour']))
cate_columns

['GarageType',
 'Exterior1st',
 'MasVnrType',
 'SaleCondition',
 'LotShape',
 'SaleType',
 'ExterQual',
 'BsmtCond',
 'MSZoning',
 'RoofStyle',
 'HouseStyle',
 'GarageFinish',
 'FireplaceQu',
 'Electrical',
 'Exterior2nd',
 'CentralAir',
 'Neighborhood',
 'BldgType',
 'PavedDrive',
 'HeatingQC',
 'BsmtQual',
 'BsmtFinType1',
 'BsmtExposure',
 'KitchenQual',
 'Foundation',
 'Alley']

In [364]:
# 数值型用中值填充
for column in num_columns:
    train[column] = train[column].fillna(train[column].median())
    test[column] = test[column].fillna(test[column].median())

# # 类别型用最多的填充 # 线上0.13488
# for column in cate_columns:
#     train[column] = train[column].fillna(train[column].mode())
#     test[column] = test[column].fillna(test[column].mode())
    
# 类别型填充'NaN' # 线上0.13436
for column in cate_columns:
    train[column] = train[column].fillna('NaN')
    test[column] = test[column].fillna('NaN')

In [365]:

trains = train[num_columns + cate_columns].copy()
tests = test[num_columns + cate_columns].copy()
Y = train[y].copy()

data = pd.concat([trains,tests],axis=0) #训练集要和测试集放一起
for column in cate_columns:
    
    t = pd.get_dummies(data[column],prefix=column)
    trains = pd.concat([trains,t[:len(train)]],axis=1)
    trains.drop(column,axis=1,inplace=True)
    
    tests = pd.concat([tests,t[len(train):]],axis=1)
    tests.drop(column,axis=1,inplace=True)

In [366]:
'''
#查看train数据连续型变量是否偏态严重
skew_var_trains = {}
for i in num_columns:
    skew_var_trains[i]=abs(trains[i].skew())
    
skew = pd.Series(skew_var_trains).sort_values(ascending=False)
skew
'''

'\n#查看train数据连续型变量是否偏态严重\nskew_var_trains = {}\nfor i in num_columns:\n    skew_var_trains[i]=abs(trains[i].skew())\n    \nskew = pd.Series(skew_var_trains).sort_values(ascending=False)\nskew\n'

In [367]:
'''
#对于train数据偏度较大的变量，进行对数转换
var_x_ln = skew[skew >= 1].index
for i in var_x_ln:
    if min(trains[i]) <= 0:
        trains[i] = np.log(trains[i] + abs(min(trains[i])) + 1)
    else:
        trains[i] = np.log(trains[i])
'''

'\n#对于train数据偏度较大的变量，进行对数转换\nvar_x_ln = skew[skew >= 1].index\nfor i in var_x_ln:\n    if min(trains[i]) <= 0:\n        trains[i] = np.log(trains[i] + abs(min(trains[i])) + 1)\n    else:\n        trains[i] = np.log(trains[i])\n'

In [368]:
'''
#检查调整后的变量偏度
skew_var_trains = {}
for i in num_columns:
    skew_var_trains[i]=abs(trains[i].skew())
    
skew = pd.Series(skew_var_trains).sort_values(ascending=False)
skew
'''

'\n#检查调整后的变量偏度\nskew_var_trains = {}\nfor i in num_columns:\n    skew_var_trains[i]=abs(trains[i].skew())\n    \nskew = pd.Series(skew_var_trains).sort_values(ascending=False)\nskew\n'

In [369]:
'''
#查看各个分类变量是否分类过多

for i in cate_columns:
    print(i,trains[i].value_counts())
'''

'\n#查看各个分类变量是否分类过多\n\nfor i in cate_columns:\n    print(i,trains[i].value_counts())\n'

In [370]:
'''
#查看test数据连续型变量是否偏态严重
skew_var_tests = {}
for i in num_columns:
    skew_var_tests[i]=abs(tests[i].skew())
    
skew = pd.Series(skew_var_tests).sort_values(ascending=False)
skew
'''

'\n#查看test数据连续型变量是否偏态严重\nskew_var_tests = {}\nfor i in num_columns:\n    skew_var_tests[i]=abs(tests[i].skew())\n    \nskew = pd.Series(skew_var_tests).sort_values(ascending=False)\nskew\n'

In [371]:
'''
#对于test数据偏度较大的变量，进行对数转换
var_x_ln = skew[skew >= 1].index
for i in var_x_ln:
    if min(tests[i]) <= 0:
        tests[i] = np.log(tests[i] + abs(min(tests[i])) + 1)
    else:
        tests[i] = np.log(tests[i])
'''

'\n#对于test数据偏度较大的变量，进行对数转换\nvar_x_ln = skew[skew >= 1].index\nfor i in var_x_ln:\n    if min(tests[i]) <= 0:\n        tests[i] = np.log(tests[i] + abs(min(tests[i])) + 1)\n    else:\n        tests[i] = np.log(tests[i])\n'

In [372]:
'''
#检查调整后的变量偏度
skew_var_tests = {}
for i in num_columns:
    skew_var_tests[i]=abs(tests[i].skew())
    
skew = pd.Series(skew_var_tests).sort_values(ascending=False)
skew
'''

'\n#检查调整后的变量偏度\nskew_var_tests = {}\nfor i in num_columns:\n    skew_var_tests[i]=abs(tests[i].skew())\n    \nskew = pd.Series(skew_var_tests).sort_values(ascending=False)\nskew\n'

In [373]:
'''
#查看各个分类变量是否分类过多

for i in cate_columns:
    print(i,tests[i].value_counts())
'''

'\n#查看各个分类变量是否分类过多\n\nfor i in cate_columns:\n    print(i,tests[i].value_counts())\n'

In [374]:
from sklearn.linear_model import Lasso,LinearRegression,Ridge,ElasticNet,TheilSenRegressor,HuberRegressor,RANSACRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools

In [349]:
X_train, X_test, y_train, y_test = train_test_split(trains, label, test_size=0.33, random_state=42)

In [375]:
regs = [
    ['Lasso',Lasso()],
    ['LinearRegression',LinearRegression()],
    ['Ridge',Ridge()],
    ['ElasticNet',ElasticNet()],
    ['TheilSenRegressor',TheilSenRegressor()],
    ['RANSACRegressor',RANSACRegressor()],
    ['HuberRegressor',HuberRegressor()],
    ['SVR',SVR(kernel='linear')],
    ['DecisionTreeRegressor',DecisionTreeRegressor()],
    ['ExtraTreeRegressor',ExtraTreeRegressor()],
    ['AdaBoostRegressor',AdaBoostRegressor(n_estimators=150)],
    ['ExtraTreesRegressor',ExtraTreesRegressor(n_estimators=150)],
    ['GradientBoostingRegressor',GradientBoostingRegressor(n_estimators=150)],
    ['RandomForestRegressor',RandomForestRegressor(n_estimators=150)],
    ['XGBRegressor',XGBRegressor(n_estimators=150)],
]

In [376]:
preds = []
for reg_name,reg in regs:
    print(reg_name)
    reg.fit(X_train,y_train)
    y_pred = reg.predict(X_test)
    if np.sum(y_pred<0) > 0:
        print('y_pred have',np.sum(y_pred<0),'negative values, we fill it with np.median(y_pred)')
        y_pred[y_pred<0] = np.median(y_pred)
    score = np.sqrt(mean_squared_error(np.log(y_test),np.log(y_pred)))
    preds.append([reg_name,y_pred])

Lasso


c:\python36\install\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LinearRegression
Ridge
ElasticNet
TheilSenRegressor
RANSACRegressor
HuberRegressor
SVR
DecisionTreeRegressor
ExtraTreeRegressor
AdaBoostRegressor
ExtraTreesRegressor
GradientBoostingRegressor
RandomForestRegressor
XGBRegressor


In [352]:
final_results = []
for comb_length in range(1,len(regs)+1):
    print('Model Amount :',comb_length)
    results = []
    for comb in itertools.combinations(preds,comb_length):
        pred_sum = 0
        model_name = []
        for reg_name,pred in comb:
            pred_sum += pred
            model_name.append(reg_name)
        pred_sum /= comb_length
        model_name = '+'.join(model_name)
        score = np.sqrt(mean_squared_error(np.log(y_test),np.log(pred_sum)))
        results.append([model_name,score])
    results = sorted(results,key=lambda x:x[1])
    for model_name,score in results:
        print(model_name,score)
    print()
    final_results.append(results[0])

Model Amount : 1
GradientBoostingRegressor 0.13531844566586
XGBRegressor 0.13710543107459716
RandomForestRegressor 0.14550378635797093
Ridge 0.16007587252975744
ExtraTreesRegressor 0.16297321712290033
Lasso 0.16480222484215487
LinearRegression 0.16557225113911905
RANSACRegressor 0.17748472495315754
ElasticNet 0.1775245272899568
HuberRegressor 0.18966490564169172
DecisionTreeRegressor 0.20485447488605063
AdaBoostRegressor 0.20765443236128012
ExtraTreeRegressor 0.23426704155247272
SVR 0.25578275787857707

Model Amount : 2
GradientBoostingRegressor+XGBRegressor 0.1344956146395033
Ridge+GradientBoostingRegressor 0.13565917313254158
Ridge+XGBRegressor 0.1362437845131906
Lasso+GradientBoostingRegressor 0.13654519378556135
Ridge+RandomForestRegressor 0.1366189109080518
GradientBoostingRegressor+RandomForestRegressor 0.13665105710325762
RandomForestRegressor+XGBRegressor 0.13666294858655184
LinearRegression+GradientBoostingRegressor 0.1367315615750738
Lasso+XGBRegressor 0.1371040646158603
Lass

Ridge+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13165071073365586
Lasso+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13166296655149823
LinearRegression+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1317054519357371
Ridge+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13408305992980468
Lasso+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13408688597565865
LinearRegression+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13413419156999284
Lasso+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegressor 0.1345478716897181
Ridge+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegressor 0.13456196730662365
LinearRegression+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegressor 0.13458374771665196
Lasso+ElasticNet+GradientBoostingRegressor+RandomForestRegressor 0.13475401744371182
Lasso+ElasticNet+RandomForestRegressor+XGBRegressor 0.1347786133895224
LinearRegression+ElasticNet+Gradient

Lasso+ElasticNet+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13300027752230395
Lasso+Ridge+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13300163918380126
LinearRegression+ElasticNet+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13302697641843414
LinearRegression+Ridge+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13305201666438837
Ridge+ElasticNet+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13309439412518384
Lasso+LinearRegression+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13330367039892538
Lasso+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13369587646366032
Lasso+Ridge+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.133712320852336
LinearRegression+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.1337268481678825
LinearRegression+Ridge+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13376580472

Ridge+ElasticNet+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor 0.1506697512397579
ElasticNet+RANSACRegressor+HuberRegressor+ExtraTreeRegressor+GradientBoostingRegressor 0.15068329664762695
Ridge+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor 0.15069029568118159
Ridge+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreesRegressor 0.15071063240359722
Lasso+Ridge+HuberRegressor+SVR+RandomForestRegressor 0.15071321280815614
Lasso+Ridge+SVR+DecisionTreeRegressor+AdaBoostRegressor 0.15072507332842253
LinearRegression+Ridge+HuberRegressor+SVR+RandomForestRegressor 0.1507494497825748
Ridge+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+XGBRegressor 0.15076055696316848
LinearRegression+Ridge+SVR+DecisionTreeRegressor+AdaBoostRegressor 0.15077207520833869
RANSACRegressor+HuberRegressor+ExtraTreeRegressor+ExtraTreesRegressor+XGBRegressor 0.15077745572844078
RANSACRegressor+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor+XGBRegressor 0.

Lasso+Ridge+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13292221651347966
LinearRegression+Ridge+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13295948547291978
Lasso+LinearRegression+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13307187765777392
Lasso+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13337557136832295
LinearRegression+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13339705378359795
Ridge+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1334969240766935
Lasso+Ridge+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13366819979876285
LinearRegression+Ridge+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+XGBRegressor 0.13370555399885614
Lasso+Ridge+ElasticNet+GradientBoostingRegre

...

Lasso+LinearRegression+Ridge+RANSACRegressor+HuberRegressor+SVR+AdaBoostRegressor+GradientBoostingRegressor 0.14665867788604836
Lasso+LinearRegression+RANSACRegressor+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.14666382159131378
Ridge+HuberRegressor+DecisionTreeRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor+XGBRegressor 0.14666494664631427
LinearRegression+Ridge+RANSACRegressor+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.14666932400332408
Lasso+ElasticNet+HuberRegressor+SVR+ExtraTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.14668943956273817
LinearRegression+ElasticNet+HuberRegressor+SVR+ExtraTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.14669184554633907
ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegress

Lasso+LinearRegression+Ridge+ElasticNet+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1339448656966355
Lasso+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13436642572555232
LinearRegression+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13439186115709842
Lasso+LinearRegression+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13441755745352169
Lasso+LinearRegression+Ridge+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13454356176501875
Lasso+LinearRegression+Ridge+ElasticNet+HuberRegressor+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13475401329895306
Lasso

Ridge+ElasticNet+RANSACRegressor+HuberRegressor+ExtraTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegressor 0.14362255060061468
Lasso+Ridge+ElasticNet+RANSACRegressor+SVR+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor+XGBRegressor 0.14363167107342004
Lasso+HuberRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.14364135007066484
Lasso+LinearRegression+HuberRegressor+SVR+DecisionTreeRegressor+ExtraTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+XGBRegressor 0.1436450573909571
LinearRegression+HuberRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.14364548042141614
Lasso+Ridge+ElasticNet+RANSACRegressor+SVR+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+RandomForestRegressor 0.14364822941107885
Lasso+LinearRegression+ElasticNet+RANSACRegressor+S

Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1348124240284061
Lasso+LinearRegression+Ridge+ElasticNet+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1349054949179472
Lasso+LinearRegression+Ridge+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13528390566094514
Lasso+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13539168921669267
Lasso+LinearRegression+ElasticNet+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13540140274958437
LinearRegression+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTrees

Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1355816025016121
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13620079077071803
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+AdaBoostRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13644968131132143
Lasso+LinearRegression+Ridge+ElasticNet+HuberRegressor+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13648385227352733
Lasso+LinearRegression+Ridge+RANSACRegressor+HuberRegressor+DecisionTreeRegressor+AdaBoostRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1366716549817215
Lasso+LinearRegression

In [353]:
final_results = sorted(final_results,key=lambda x:x[1])
for model_name,score in final_results:
    print(model_name,score)

Ridge+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13165071073365586
Ridge+GradientBoostingRegressor+XGBRegressor 0.132420139998821
Lasso+Ridge+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13284697474169876
Lasso+Ridge+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13292221651347966
Lasso+ElasticNet+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13300027752230395
Lasso+LinearRegression+Ridge+ElasticNet+DecisionTreeRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.13354607053987774
Lasso+LinearRegression+Ridge+ElasticNet+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XGBRegressor 0.1339448656966355
GradientBoostingRegressor+XGBRegressor 0.1344956146395033
Lasso+LinearRegression+Ridge+ElasticNet+RANSACRegressor+DecisionTreeRegressor+ExtraTreesRegressor+GradientBoostingRegressor+RandomForestRegressor+XG

In [354]:
[b for b in zip(itertools.count(),[a[0] for a in regs])]

[(0, 'Lasso'),
 (1, 'LinearRegression'),
 (2, 'Ridge'),
 (3, 'ElasticNet'),
 (4, 'RANSACRegressor'),
 (5, 'HuberRegressor'),
 (6, 'SVR'),
 (7, 'DecisionTreeRegressor'),
 (8, 'ExtraTreeRegressor'),
 (9, 'AdaBoostRegressor'),
 (10, 'ExtraTreesRegressor'),
 (11, 'GradientBoostingRegressor'),
 (12, 'RandomForestRegressor'),
 (13, 'XGBRegressor')]

In [355]:
pred = np.mean(list(map(lambda x:regs[x][1].predict(tests),[0,12,13])),axis=0)

In [356]:
sub = pd.DataFrame({'Id':id,'SalePrice':pred})
sub.to_csv('submission_Universe_fillNaN3.csv',index=None)